#  Fake news detection
   <p>This is a rough general analysis on the dataset, done inorder to get an overall  understanding of the processes required and to get an idea on choosing the right path for the analysis. I used common techniques of NLP inorder to vectorize and understand the word distribution in the text involved. tokenization, removing stop words, lemmatization and stemming are done. For the model part, a pipeline is used for fitting the training data on TfidfVectorizer and knn algorithm. A test accuracy of Test Accuracy: 0.851 is obtained.  </p>
   
   <p>For further analysis, techniques like latent dirichlet allocation, gridsearch or randomsearch, other ML algorithms, SMOTE, and so on.</p>

## Loading the data

In [ ]:
# importing the libraries
import pandas as pd
import numpy as np

# NLP
import nltk
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

# BoW
from sklearn.feature_extraction.text import TfidfVectorizer

# model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [ ]:
# loading the data from the downloaded csv file
data = pd.read_csv('Fake_Real.csv')

In [ ]:
# checking the data
data.info()

In [ ]:
data.head()

In [ ]:
# dropping the redundant 'Unnamed: 0' column
data.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
# printing the head to confirm the previous step
data.head()

In [ ]:
# getting the counts of 'target' values
data['target'].value_counts()

In [ ]:
# getting the count of various 'subject' in the news dataset
data['subject'].value_counts()

In [ ]:
# checking the shape of the dataset
data.shape

In [ ]:
# dropping the na values
data.dropna(inplace=True)

In [ ]:
# checking the data to confirm the drop of na values
data.shape

## Data Cleaning and extraction

In [ ]:
# Clean text data
data.loc[0,'text'][:1000]

In [ ]:
# def preprocess_text(text):
#     # Tokenize
#     tokens = word_tokenize(text.lower())
# Remove stop words
# filtered_tokens = [token for token in tokens if ((token not in stopwords.words('english')) and (not token.isdigit()))]
# filtered_tokens = [token for token in filtered_tokens if ((token not in  set(string.punctuation)))]

#     # Lemmatize the tokens
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
#     # Join the tokens back into a string
#     processed_text = ' '.join(lemmatized_tokens)
#     #stem the tokens
#     porter = PorterStemmer()
#     clean_text = " ".join(porter.stem(token) for token in processed_text.split())
#     return processed_text

In [ ]:
# data['text'] = data['text'].apply(preprocess_text)
# data.head()

In [ ]:
# Tokenize and removing stop words
stop_words = set(stopwords.words('english'))
def clean_text(text):
    # Tokenize
    tokens = word_tokenize(text)
    # remove non-alphabetical characters and stopwords
    cleaned_tokens = [re.sub(r'[^a-zA-Z ]', '', text).lower() for text in tokens if text.lower() not in stop_words]
    cleaned_tokens = [token for token in cleaned_tokens if ((token not in  set(string.punctuation)))]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in cleaned_tokens]
    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    #stem the tokens
    porter = PorterStemmer()
    cleaned_text = " ".join(porter.stem(token) for token in processed_text.split())
    return cleaned_text

# Apply the function across the DataFrame
data['cleaned_text'] = data['text'].apply(clean_text)

#print(data['cleaned_text'][1])

In [ ]:
print(data['cleaned_text'][1])

## Model building

In [ ]:
# defining the X and y 
X = data['cleaned_text'].values
y = data['target'].values
X.shape,y.shape
# splitting data to train-test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=22)

In [ ]:
# BoW model
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

In [ ]:
# defining the knn model
knn = KNeighborsClassifier(n_neighbors = 3)

In [ ]:
# defining the pipeline to fit the training data
knn_tfidf = Pipeline([
    ('vect',tfidf),
    ('knn',knn)
])
knn_tfidf.fit(X_train,y_train)

In [ ]:
# printing the accuracy of the model
print(f'Test Accuracy: {knn_tfidf.score(X_test, y_test):.3f}')